<a name="top"></a>
<div style="width:1000 px">

<div style="float:right; width:98 px; height:98px;">
<img src="https://raw.githubusercontent.com/Unidata/MetPy/master/src/metpy/plots/_static/unidata_150x150.png" alt="Unidata Logo" style="height: 98px;">
</div>

<h1>Units in MetPy</h1>
<h3>Unidata AMS 2021 Student Conference</h3>

<div style="clear:both"></div>
</div>

---

This notebook will help you become familiar with unit support in the `MetPy` library. Learn how to apply units to values, calculate new values from united values, and use units in `MetPy` functions. `MetPy` provides unit support through the `Pint` library. You can access the documentation for `Pint` by clicking on the image to the right.
<a href=https://pint.readthedocs.io/en/stable/><div style="float:right; width:250 px"><img src="https://pint.readthedocs.io/en/stable/_images/logo-full.jpg" alt="logo for the Pint library" style="height: 300px;"></div></a>


### Focuses
* Introduce the use of `units` to track values with physical units
* Demonstrate basic operations with and on values with units
* Show examples of using units in `MetPy`


### Objectives
1. [Introduction to Units](#1.-Introduction-to-Units)
1. [Quantity Methods and Attributes](#2.-Quantity-Methods-and-Attributes)
1. [MetPy and Units](#3.-MetPy-and-Units)
1. [Gotchas](#4.-Gotchas)
---

### Imports

In [ ]:
import numpy as np
import metpy.calc as mpcalc
from metpy.units import units

---

## 1. Introduction to Units


`MetPy` provides units support through the `Pint` library. Values with units attached to them (quantities) are described by the [`Pint` documentation](https://pint.readthedocs.io/en/stable/) as "the product of a numerical value and a unit of measurement." They are used to programatically keep track of physical quantities so we can worry less about unit conversion errors and focus on our coding objective instead.

### Attaching Units to Numerical Values
We can apply units to a numerical value by multiplying that value by the units we desire in a couple of different ways -- either accessing the units as an attribute of `units` or passing a string to `units` as an argument. Either way, we get the same result:

In [ ]:
distance = 5. * units.meter
distance

In [ ]:
distance = 5. * units('meter')
distance

Alternatively, we can use the `Quantity` constructor to do the same thing.

In [ ]:
distance = units.Quantity(5., 'meter')
distance

We are not limited to applying units to scalars. They work with `numpy` arrays as well!

In [ ]:
distance = np.arange(1., 11.) * units.meter
distance

If our units are not as simple as e.g., distance, we can chain together unit assignments. Say we have a vertical acceleration that we want to apply units to. We can multiply by meters, then divide by seconds twice. Alternatively, we can express this as a string of words, or even a string of abbreviations.

In [ ]:
accel = np.arange(10) * units.meter / units.second / units.second
accel

In [ ]:
accel = np.arange(10) * units('meter / second ** 2')
accel

In [ ]:
accel = np.arange(10) * units('m / s ** 2')
accel

### Calculations with Units

Once units are attached to a quantity, we can do math with other united quantities.

In [ ]:
x = np.random.randint(25, size=(5, 5)) * units.meter
y = np.random.randint(25, size=(5, 5)) * units.meter

z = x - y
z

If the dimensionality of the quantities we are operating on don't match, we will get a `DimensionalityError`. For example, try uncommenting the last line in the following cell and then running the cell.

In [ ]:
distance = np.random.random((5, 5)) * units.meter
time = np.random.random((5, 5)) * units.second

# velocity = distance - time

In the case that the unit dimensionalities are different, but the mathematical operation makes it valid, we won't get an error. Instead we get a quantity with new units, as we expect.

In [ ]:
distance = np.random.random((5, 5)) * units.meter
time = np.random.random((5, 5)) * units.second

velocity = distance / time
velocity

<a href="#top">Top</a>

---

## 2. Quantity Methods and Attributes

When we multiply a value by units, what we are actually doing is building a `Quantity` object. A `Quantity` has many useful methods and attributes to manipulate its value and give us information about it. Some of the more common and useful ones are:
* `.magnitude`, and `.m`
* `.to()`
* `.units`

Say we have a 1-foot long ruler that measures distance in whole inches.

In [ ]:
ruler = np.arange(1.0, 13.0) * units.inch
ruler

Perhaps we want to go rogue and create a ruler without units. We can get the *magnitude* of the ruler with the `.magnitude` attribute (or shorthand `.m`).

In [ ]:
ruler.m

But that probably isn't a great idea. Instead, let's convert the units of the ruler to measure in centimeters. We do this using the `.to` method.

In [ ]:
ruler = ruler.to('centimeter')
ruler

After working with our ruler for awhile, maybe we forget what units are attached to it. We can check with the `.units` attribute.

In [ ]:
ruler.units

<a href="#top">Top</a>

---

## 3. MetPy and Units

Many of the functions in `MetPy` expect their arguments to have units. Though it may seem like slightly more effort, requiring units saves time for everyone. Let's look at a simple example with u and v wind components. Say we want to calculate wind speed and wind direction from these components.

In [ ]:
u = np.array([5., 3., 7., 2., -9.])
v = np.array([-2., 5., -7., 3., 2.])

If we try to calculate wind speed using `mpcalc.wind_speed` without applying units, we will get a `ValueError` and an explanation for how to apply units to our values. For example, try uncommenting the line in the following cell and running the cell.

In [ ]:
# mpcalc.wind_speed(u, v)

Notice that at the bottom of the error message, an explanation is provided that `u` needs `"[speed]"`. This is not a specific unit, but instead a dimensionality. We can apply any units we like to `u` and `v` as long as their dimensionality is speed. Let's use meter / second.

In [ ]:
u = u * units("meter / second")
v = v * units("meter / second")
wind_speed = mpcalc.wind_speed(u, v)
wind_speed

Great! Now let's get the wind direction with `mpcalc.wind_direction`

In [ ]:
wind_dir = mpcalc.wind_direction(u, v)
wind_dir

Notice that in both cases the results have units attached to them, and `wind_dir` has units of `degree` as we expect.

<a href="#top">Top</a>

---

## 4. Gotchas


There are a few places that one can easily get tripped up when using units in an atmospheric science context. Additionally, sometimes units don't play nicely with the data type we are working with, so we need to handle them in a different way.

### Millibars versus millibarns

We will commonly work with pressure values that have units of millibars, or mb. However, the abbreviation mb means something different to `Pint`...

In [ ]:
pressure = 500. * units.mb
print(pressure)

A millibarn is a unit of area, which is likely not what we're looking for. To avoid this, we want to spell out millibar when using it as a physical unit.

In [ ]:
pressure = 500. * units.millibar
print(pressure)

### Temperature

Sometimes temperature is expressed with units of a *difference*, or a *delta*. These are treated as different units, but can interact with their "whole" counterparts. You can read more about temperature conversion [here.](https://pint.readthedocs.io/en/stable/nonmult.html) Otherwise, here are a few examples to consider.

In [ ]:
delta_t = 6. * units.delta_degF
temperature = 72. * units.degF
temperature + delta_t

In [ ]:
temperature.to('kelvin') + delta_t

In [ ]:
10. * units('degF') - 1. * units('kelvin')

### Attaching Units to Masked Arrays

Finally, some data types don't play with units as nicely as we would like them to. A well-known and common example of this occurs with `numpy.ma.array`, i.e., `numpy` masked arrays. Instead of multiplying a masked array by the units we desire, we need to call the `Quantity` constructor directly.

In [ ]:
mask = [False, True, False, False, True]
values = [87., 105., 94., 45., 107.]

humidity = units.Quantity(np.ma.array(values, mask=mask), 'percent')
humidity

<a href="#top">Top</a>

---

## See also

If you need more examples of working with units, check out the [MetPy docs](https://unidata.github.io/MetPy/latest/index.html), [MetPy example gallery](https://unidata.github.io/MetPy/latest/examples/index.html), and [Pint docs](https://pint.readthedocs.io/en/stable/index.html).

If you feel like you need a bigger challenge, consider working on one or both of the following:
* [Isentropic Analysis Case Study](https://unidata.github.io/pyaos-ams-2021/projects/isentropic_analysis.html)
* [Severe Weather Outbreak Case Study](https://unidata.github.io/pyaos-ams-2021/projects/severe_wx_outbreak.html)

<a href="#top">Top</a>

---